In [1]:
import os

In [2]:
%pwd


'c:\\Users\\User\\Desktop\\chicken Diesease\\Chicken_Diesease_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd


'c:\\Users\\User\\Desktop\\chicken Diesease\\Chicken_Diesease_Classification'

In [9]:
pip install ensure


  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached six-1.17.0-py2.py3-none-any.whl (11 kB)

   ---------------------------------------- 2/2 [ensure]

Note: you may need to restart the kernel to use updated packages.


In [10]:
from ensure import ensure_annotations
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      


In [14]:
import os
import urllib.request as request
import zipfile
from CNNClassifier import logger
from CNNClassifier.utils.common import get_size

In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [21]:
try:
    # Initialize configuration manager
    config_manager = ConfigurationManager()
    
    # Get data ingestion config
    data_ingestion_config = config_manager.get_data_ingestion_config()
    
    # Initialize data ingestion with the config
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    # Download the dataset
    data_ingestion.download_file()
    
    # Extract the zip file
    data_ingestion.extract_zip_file()
    
    print("Data ingestion completed successfully!")

except Exception as e:
    # Log the error (if you have logger)
    logger.error(f"Error during data ingestion: {e}")
    raise e


[2025-10-16 16:00:34,038: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-16 16:00:34,042: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-16 16:00:34,045: INFO: common: created directory at: artifacts]
[2025-10-16 16:00:34,050: INFO: common: created directory at: artifacts/data_ingestion]
[2025-10-16 16:00:44,945: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 15556855
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "de15f351dbcf48a3cae28c61dda8332f831cf9870073bf03e5da8aa179ee43d7"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6D83:2AFD13:CB6AA:1EC480:68F0C94A
Accept-Ranges: bytes
Date: Thu, 16 Oct 2025 10:30:36 GMT
Via: 1.1 varnish
X-Served-By: cache-sin-wsss1830041-SIN
X-C